In [ ]:
from bs4 import BeautifulSoup, NavigableString, Tag
import requests
import re
import pandas as pd
import numpy as np

# Modularized version

In [ ]:
class scrapper():
  def __init__(self, *args) -> None: #First argument is start page num second is end page number
    assert args[0] >= 0 and args[0] <= 3
    assert len(args) == 1 or (args[1] > args[0] and args[1] <= 3)
    self.alread_run = False
    self.souplst = []
    self.scraped = {'details':[],'resources':[],'prior_acts_0':[],'prior_acts_re_0':[],'prior_acts_type_0':[]}
    templete = 'https://www.sec.gov/rules/rulemaking-activity?page='

    if len(args) == 1:
      self.souplst.append(self.getsoup(templete+str(args[0])))
    else:
      for i in range(args[0],args[1]+1):
        self.souplst.append(self.getsoup(templete+str(i)))

  def getsoup(self, url):
    response = requests.get(url)
    return BeautifulSoup(response.text)

  def linkextract(self):
    if self.alread_run:
      return print('Links aleady extracted!')

    self.rulelinks = list()
    for i, soup in enumerate(self.souplst):
      table_rows = soup.find('table',{'aria-describedby' : "sr-only-rulemaking_activity_page"}).findAll('tr')
      self.souplst[i] = table_rows #Replace each element in list by a list of table rows

    for soup in self.souplst:
      for tr in soup[1:]:
        url = tr.find('a',{'href':True})['href']
        self.rulelinks.append('https://www.sec.gov' + url)

    self.alread_run = True

  def detail_scrape(self):

    for link in self.rulelinks:
      page = self.getsoup(link)
      lst = list(page.find('div',{'class':'side-details'}))
      string = ''
      for tag in lst:
          if type(tag) is Tag:
              string += tag.get_text()

      self.scraped['details'].append(string)

  def resource_scrape(self):

    for link in self.rulelinks:
      page = self.getsoup(link)
      div = page.find('div',{'class':'side-resources'})
      if div:
        a = list(div)
      else:
        self.scraped['resources'].append(np.nan)
        continue

      href = ''
      for tag in a:
        if tag.name == 'div':
          href += tag.find('a')['href']
          href += '\n'

      self.scraped['resources'].append(href)

  def prior_act(self):

    for link in self.rulelinks:
      page = self.getsoup(link)
      sections = page.find('div',{'class':"effect-none"}).findAll('div',{'class':"details-section"})
      h3tags = page.find('div',{'class':"effect-none"}).findAll('h3')

      #If no prior actions detected, fill the row with Nan value and skip to next rule link
      if len(sections) == 0:
        num_prior_acts_cols = (len(self.scraped) - 2) // 3
        for i in range(num_prior_acts_cols):
          self.scraped['prior_acts_'+str(i)].append(np.nan)
          self.scraped['prior_acts_re_'+str(i)].append(np.nan)
          self.scraped['prior_acts_type_'+str(i)].append(np.nan)

        continue


      #Iterate scraping for the number of prior act sections
      for j, section in enumerate(sections):
        a_text = h3tags[j].find('a').get_text()
        string_detail = ''
        string_resource = ''

        for tag in list(section):
          if type(tag) == Tag and tag['class'][0] != 'side-resources':
            for span in tag.findAll('span'):
              if span.get_text() == 'Federal Register Publish Date':
                string_detail += 'Federal Register Publish Date'
                string_detail += '\n'
                string_detail += span.find_next_sibling().get_text()
              else:
                string_detail += span.get_text()
                string_detail += '\n'


          elif type(tag) == Tag and tag['class'][0] == 'side-resources':
            for a in tag.findAll('a'):
              string_resource += a['href']
              string_resource += '\n'

        if 'prior_acts_'+str(j) in self.scraped:
          self.scraped['prior_acts_'+str(j)].append(string_detail)
          self.scraped['prior_acts_re_'+str(j)].append(string_resource)
          self.scraped['prior_acts_type_'+str(j)].append(a_text)

        else:
          num_rows = len(self.scraped['prior_acts_0'])
          self.scraped['prior_acts_'+str(j)] = [np.nan for i in range(num_rows-1)] #Creating another column with the same height of the adjacent one
          self.scraped['prior_acts_re_'+str(j)] = [np.nan for i in range(num_rows-1)] #When the column is appended, default values are populated with Nan
          self.scraped['prior_acts_type_'+str(j)] = [np.nan for i in range(num_rows-1)]

          self.scraped['prior_acts_'+str(j)].append(string_detail)
          self.scraped['prior_acts_re_'+str(j)].append(string_resource)
          self.scraped['prior_acts_type_'+str(j)].append(a_text)

      num_prior_acts_cols = (len(self.scraped) - 2) // 3
      if len(sections) < num_prior_acts_cols:
        for k in range(len(sections), num_prior_acts_cols):
          self.scraped['prior_acts_'+str(k)].append(np.nan)
          self.scraped['prior_acts_re_'+str(k)].append(np.nan)
          self.scraped['prior_acts_type_'+str(k)].append(np.nan)


In [ ]:
scrape = scrapper(0,3)
scrape.linkextract()
scrape.detail_scrape()
scrape.resource_scrape()
scrape.prior_act()

# Non-modularized version

In [ ]:
scraped = {'details':[],'resources':[],'prior_acts_0':[],'prior_acts_re_0':[]}

In [ ]:
#Rule links extraction

page = requests.get('https://www.sec.gov/rules/rulemaking-activity?page=0')

table = BeautifulSoup(page.text)
table_rows = table.find('table',{'aria-describedby' : "sr-only-rulemaking_activity_page"}).findAll('tr')
rulepage_urls = []

for tr in table_rows[1:]:
  url = tr.find('a',{'href':True})['href']
  rulepage_urls.append('https://www.sec.gov' + url)
rulepage_urls

['https://www.sec.gov/rules/2023/02/safeguarding-advisory-client-assets',
 'https://www.sec.gov/rules/2022/05/private-fund-advisers-documentation-registered-investment-adviser-compliance-reviews',
 'https://www.sec.gov/rules/2015/03/exemption-certain-exchange-members',
 'https://www.sec.gov/rules/2023/07/s7-13-23',
 'https://www.sec.gov/rules/2023/07/s7-12-23',
 'https://www.sec.gov/rules/2022/03/cybersecurity-risk-management-strategy-governance-and-incident-disclosure',
 'https://www.sec.gov/rules/2023/07/s7-11-23',
 'https://www.sec.gov/rules/2021/12/money-market-fund-reforms',
 'https://www.sec.gov/rules/2021/12/position-reporting-large-security-based-swap-positions',
 'https://www.sec.gov/rules/2023/06/33-11205',
 'https://www.sec.gov/rules/2022/03/removal-references-credit-ratings-regulation-m',
 'https://www.sec.gov/rules/2021/12/prohibition-against-fraud-manipulation-and-deception-connection-security-based-swaps',
 'https://www.sec.gov/rules/2023/05/33-11197',
 'https://www.sec.

In [ ]:
#Detail section
for link in rulepage_urls:
  page = requests.get(link)
  page = BeautifulSoup(page.text)

  lst = list(page.find('div',{'class':'side-details'}))
  string = ''
  for tag in lst:
      if type(tag) is Tag:
          string += tag.get_text()

  scraped['details'].append(string)

Details
Public Comments Due
Oct. 2, 2023

File Number
S7-13-23

Rule Type

Proposed


Release Number
IA-6354 

SEC Issue Date
July 26, 2023

Federal Register Publish Date
Aug. 1, 2023



In [ ]:
# Resource section
for link in rulepage_urls:
  page = requests.get('https://www.sec.gov/rules/2023/07/s7-13-23')
  page = BeautifulSoup(page.text)

  string = ''
  a = list(page.find('div',{'class':'side-resources'}))
  for tag in a:
    if tag.name == 'div':
      string += tag.find('a')['href']
      string += '\n'
  scraped['resources'].append(string)

/files/rules/proposed/2023/ia-6354.pdf
https://www.federalregister.gov/documents/2023/08/01/2023-16287/exemption-for-certain-investment-advisers-operating-through-the-internet
/files/ia-6354-fact-sheet.pdf
https://www.sec.gov/news/press-release/2023-141



In [ ]:
# Prior Action

page = requests.get('https://www.sec.gov/rules/2017/12/investment-company-reporting-modernization')
page = BeautifulSoup(page.text)

sections = page.find('div',{'class':"effect-none"}).findAll('div',{'class':"details-section"}) #It will be list already

strings = list()
links = dict()
for i, section in enumerate(sections):
  strings.append('')
  links[i] = list()
  for tag in list(section):
    if type(tag) == Tag and tag['class'][0] != 'side-resources':
      for span in tag.findAll('span'):
        if span.get_text() == 'Federal Register Publish Date':
          strings[-1] += 'Federal Register Publish Date'
          strings[-1] += '\n'
          strings[-1] += span.find_next_sibling().get_text()
        else:
          strings[-1] += span.get_text()
          strings[-1] += '\n'
    elif type(tag) == Tag and tag['class'][0] == 'side-resources':
      for a in tag.findAll('a'):
        links[i].append(a['href'])

strings

['Release Number\n33-10577\nSEC Issue Date\nNov. 29, 2018\nFederal Register Publish DateDec. 4, 2018',
 'Release Number\n33-10442\n33-9776\n33-10231\n34-82241\nIC-32936\nSEC Issue Date\nDec. 8, 2017\nFederal Register Publish DateDec. 14, 2017RIN\n3235-AL42\n',
 'Release Number\n33-10231\n33-9776, 33-10442, 34-79095\nIC-32314\nSEC Issue Date\nOct. 13, 2016\nFederal Register Publish DateNov. 18, 2016RIN\n3235-AL42\n',
 'Release Number\n33-9776\n33-10231\n34-75002\nIC-31610\nSEC Issue Date\nMay 20, 2015\nFederal Register Publish DateJune 12, 2015RIN\n3235-AL42\n',
 'Release Number\n33-9776\n33-10231\n34-75002\nIC-31610\nSEC Issue Date\nMay 20, 2015\nFederal Register Publish DateJune 12, 2015RIN\n3235-AL42\n']

# To dataframe

In [ ]:
df = pd.DataFrame(scrape.scraped)

In [ ]:
df.to_csv('df.csv',index=False)

In [ ]:
df

,details,resources,prior_acts_0,prior_acts_re_0,prior_acts_type_0,prior_acts_1,prior_acts_re_1,prior_acts_type_1,prior_acts_2,prior_acts_re_2,...,prior_acts_type_5,prior_acts_6,prior_acts_re_6,prior_acts_type_6,prior_acts_7,prior_acts_re_7,prior_acts_type_7,prior_acts_8,prior_acts_re_8,prior_acts_type_8
0,Details\nPublic Comments Due\n60 days after pu...,/files/rules/proposed/2023/ia-6240.pdf\nhttps:...,Release Number\nIA-6240\nSEC Issue Date\nFeb. ...,/files/rules/proposed/2023/ia-6240.pdf\nhttps:...,Proposed Rule (IA-6240),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Details\nFile Number\nS7-03-22\n\nRule Type\n\...,/files/rules/proposed/2022/34-94868.pdf\nhttps...,Release Number\n34-94868\nSEC Issue Date\nMay ...,/files/rules/proposed/2022/34-94868.pdf\nhttps...,Proposed Rule (34-94868),Release Number\nIA-5955\n34-94868\nSEC Issue D...,/files/rules/proposed/2022/ia-5955.pdf\nhttps:...,Proposed Rule (IA-5955),NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Details\nFile Number\nS7-05-15\n\nRule Type\n\...,/files/rules/proposed/2022/34-95388.pdf\nhttps...,Release Number\n34-95388\nSEC Issue Date\nJuly...,/files/rules/proposed/2022/34-95388.pdf\nhttps...,Proposed Rule (34-95388),Release Number\n34-74581\nSEC Issue Date\nMarc...,/files/rules/proposed/2015/34-74581.pdf\nhttps...,Proposed Rule (34-74581),NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Details\nPublic Comments Due\nOct. 2, 2023\n\n...",/files/rules/proposed/2023/ia-6354.pdf\nhttps:...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Details\nPublic Comments Due\nOct. 10, 2023\n\...",/files/rules/proposed/2023/34-97990.pdf\nhttps...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1114,Details\nRule Type\n\nInterpretive\n\n\nReleas...,/files/rules/interp/1973/34-10429.pdf\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1115,Details\nRule Type\n\nInterpretive\n\n\nReleas...,/files/rules/interp/1973/33-5347.pdf\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1116,Details\nRule Type\n\nInterpretive\n\n\nReleas...,/files/rules/interp/1971/33-5211.pdf\n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1117,Details\nRule Type\n\nFinal\n\n\nRelease Numbe...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
s = df['details'].str.extract(r'Public Comments Due\n([\w\d. ,]*\d{4,})')